# PHASE 3: PROFILE + GEN Q'S

In [1]:
import utils.csv_utils as csv_utils 
import utils.dir_utils as dir_utils
import utils.dict_utils as dict_utils 
import utils.ptr_utils as ptr_utils
import utils.constants as constants 
import helpers.official as official
import helpers.search as search
import helpers.congress as congress
import pandas as pd

In [3]:
_, input_df = dir_utils.get_data(combined=True)
    
sector_df = dir_utils.get_mapping(sector=True)
industry_df = dir_utils.get_mapping(industry=True)

# {canonical_name_input_based : link, ...}
input_all_officials_name = {}
# {link : canonical_name_input_based, ....} 
input_all_officials_link = {} 
# (canonical_name_input_based, ...)
names = set()

for _,t in input_df.iterrows():        
    name = official.get_name(t)
    if name not in names:    
        link = search.get_wiki_link(name)        
        input_all_officials_link = dict_utils.increment_dictionary(input_all_officials_link, link, name, not_math=True)
        input_all_officials_name = dict_utils.increment_dictionary(input_all_officials_name, name, link, not_math=True)
        names.add(name)

# {link : (canonical_name_input_based, official_object), ... }
input_all_officials_objects = {}
for link, person in input_all_officials_link.items(): 
    off = search.wiki_search(person)      
    input_all_officials_objects[link] = (person, off)

# 17m 42.7s

In [7]:
# zero = pd.read_csv("../results/csv/profit/sample/b_0.csv")
# seven = pd.read_csv("../results/csv/profit/sample/b_7.csv")
# fourteen = pd.read_csv("../results/csv/profit/sample/b_14.csv")

# # copying people from certain committees?

# def t_to_obj(t):
#     name = t['name']
#     link = input_all_officials_name[name]
#     _, obj = input_all_officials_objects[link]
#     return obj

In [49]:
import numpy

def mean1(d):
    for k,v in d.items():
        if v != 0 and v != []:
            # print(v)
            # print(numpy.average(v))
            d[k] = round(float(numpy.average(v)), 2)
        
    return d 


def z(group):
    p26 = {}
    p52 = {}
    p104 = {}

    for _, t in group.iterrows():
        obj = t_to_obj(t)
        # committees = obj.get_asgts()
        sector = dir_utils.search_mapping(sector_df, t['ticker_purch'], sector=True)
        # for i in committees:
        #     i = i [ : i.find("(")]
        #     if 'Subcommittee' not in i:
        if ptr_utils.isvalid(t['p_104']): 
            p104 = dict_utils.increment_list_in_dictionary(p104, sector, float(t['p_104']))
        if  ptr_utils.isvalid(t['p_52']): 
            p52 = dict_utils.increment_list_in_dictionary(p52, sector, float(t['p_52']))
        if  ptr_utils.isvalid(t['p_26']): 
            p26 = dict_utils.increment_list_in_dictionary(p26, sector, float(t['p_26']))
         
    p26 = mean1(p26)
    p52 = mean1(p52) 
    p104 = mean1(p104)
            
    return p26, p52, p104



In [50]:
p26, p52, p104 = z(zero)
csv_utils.make_csv_multiple_dicts('/', "b_0_sector", (p26, p52, p104), ['', 'p26', 'p52', 'p104'])
p26, p52, p104 = z(seven)
csv_utils.make_csv_multiple_dicts('/', "b_7_sector", (p26, p52, p104), ['', 'p26', 'p52', 'p104'])
p26, p52, p104 = z(fourteen)
csv_utils.make_csv_multiple_dicts('/', "b_14_sector", (p26, p52, p104), ['', 'p26', 'p52', 'p104'])

In [ ]:
# {link : (canonical_name_input_based, official_object), ... }
questionable_people_objects = {}
indexes_to_keep = [] 

for index,t in input_df.iterrows():
    if ptr_utils.isvalid(t[constants.TICKER]):
        name = official.get_name(t)
        year = str(ptr_utils.get_year((t[constants.TDATE])))
        sector = dir_utils.search_mapping(sector_df, t[constants.TICKER], sector=True)
        obj = t_to_obj(t)
        committees = obj.get_asgts()
        
        for comm in committees:
            comm_year = official.get_committee_year(comm)                
            if sector in comm and year in comm_year: 
                link = input_all_officials_name[name]
                questionable_people_objects[link] =  (name, obj)
                indexes_to_keep.append(index)
                break
        
        if index not in indexes_to_keep:
            input_df.drop(index, inplace=True)
    else:
        input_df.drop(index, inplace=True)

input_df.to_csv("questionable.csv", sep='\t')

## Profile 

In [ ]:
# s = set()
# i = 0 
# for _,t in input_df.iterrows():
#     if t[constants.TDATE] == '2020/04/02' and t[constants.TYPE] == 'Purchase':
#         # sector = dir_utils.search_mapping(sector_df, t[constants.TICKER], sector=True)
#         # if sector == 'Financial Services':
#         i += 1
#         s.add(t_to_obj(t).get_label())
#             # print()
#             # s.add(t['ticker'])

# print(s)
# print(i)

### Committees

In [ ]:
def profile_committee(group, normalized=None):
    # {party_and_gender : 45_ppl}
    d = {}
        
    for _, (_, obj) in group.items(): 
        for a in obj.get_asgts():
            a = a[ : a.find(" (")]
            d = dict_utils.increment_dictionary(d, a)

    # Normalize
    if normalized: 
        d = dict_utils.normalize(d, normalized, percent=True)

    return d

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/profile/committees")

d1 = profile_committee(questionable_people_objects)
csv_utils.make_csv(dir, "profile_committee", d1, ["committee", constants.INPUT])

d3 = profile_committee(questionable_people_objects, profile_committee(input_all_officials_objects))
csv_utils.make_csv(dir, "profile_committee_normalized", d3, ["committee", constants.INPUT])

### Party and Gender

In [ ]:
def profile_party_and_gender(group, normalized=None):
    # {party_and_gender : 45_ppl}
    d = {}
        
    for link, (name, obj) in group.items(): 
        grouped = obj.party + ", " + official.get_gender(name, link=link)
        d = dict_utils.increment_dictionary(d, grouped)

    # Normalize
    if normalized: 
        d = dict_utils.normalize(d, normalized, percent=True)

    return d

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/profile/partyandgender")

d1 = profile_party_and_gender(questionable_people_objects)
csv_utils.make_csv(dir, "profile_party_and_gender", d1, [constants.PARTY + ", " + constants.GENDER, constants.INPUT])

d3 = profile_party_and_gender(questionable_people_objects, profile_party_and_gender(input_all_officials_objects))
csv_utils.make_csv(dir, "profile_party_and_gender_normalized", d3, [constants.PARTY + ", " + constants.GENDER, constants.INPUT])

### Age (Lowest, Highest, Average)

In [ ]:
def profile_age(group):
    # {age, age, ...}
    l = []
    
    # group = {link : (canonical_name_input_based, official_object) ... }
    for _, (_, off_obj) in group.items(): 
        l.append(off_obj.get_age())
    
    l.sort()
    
    d = {}
    d["0. Youngest"] = l[0]
    d["1. Average"] = round((sum(l) / len(l)))
    d["2. Oldest"] = l[len(l)-1]
    
    return d 
    
d1 = profile_age(questionable_people_objects)
dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/profile/age")
csv_utils.make_csv(dir, "profile_age", d1, ["Age", constants.INPUT])


def profile_age_2(group, normalized=None):
    # {age : #_of_people, ...}
    d = dict(constants.age_groups)
    
    for (_, off_obj) in group.values(): 
        age = off_obj.get_age()
        d = dict_utils.increment_dictionary(d, ptr_utils.which_age_group(age)) 
    
    if normalized: 
        d_prime = dict_utils.normalize(d, normalized, percent=True)
        return d, d_prime
        
    return d 

d1, d4 = profile_age_2(questionable_people_objects, profile_age_2(input_all_officials_objects))

csv_utils.make_csv(dir, "profile_age_2", (d1), ["Age", constants.INPUT])
csv_utils.make_csv(dir, "profile_age_2_normalized", (d4), ["Age", constants.INPUT])


def profile_age_3(group):
    # {age: freq_count , .... }
    d = {}
    
    # group = {link : (canonical_name_input_based, official_object) ... }
    for _, (_, off_obj) in group.items(): 
        d = dict_utils.increment_dictionary(d, off_obj.get_age())
    
    return d 
    
d3 = profile_age_3(questionable_people_objects)

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/profile/age")
csv_utils.make_csv(dir, "profile_age_3", (d3), ["Age", constants.INPUT])


def profile_age_box(group):
    # {age, age, ...}
    l = []
    
    # group = {link : (canonical_name_input_based, official_object) ... }
    for _, (_, off_obj) in group.items(): 
        l.append([off_obj.get_age()])
        
    return l 
    
d1 = profile_age_box(questionable_people_objects)

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/profile/age")
csv_utils.make_csv_base(dir, "profile_age_box_input", ["Age_Input"], d1)

### Oldest and Most Recent Dates (transaction and disclosure)

In [ ]:
def profile_dates(group, type):
    lowest_tdate = lowest_ddate = highest_tdate = highest_ddate = None
    lowest_tdate_obj = lowest_ddate_obj = highest_tdate_obj = highest_ddate_obj = None

    for _,t in group.iterrows():
        curr = t[constants.TDATE]            
        if not lowest_tdate or curr < lowest_tdate:
            lowest_tdate = curr 
            lowest_tdate_obj = t
        if not highest_tdate or curr > highest_tdate:
            highest_tdate = curr  
            highest_tdate_obj = t

        curr = t[constants.DDATE]
        if not lowest_ddate or curr < lowest_ddate:
            lowest_ddate = curr 
            lowest_ddate_obj = t
        if not highest_ddate or curr > highest_ddate:
            highest_ddate = curr 
            highest_ddate_obj = t

    print("Oldest transaction_date for {}: {} by {} \n {}".format(type, lowest_tdate, official.get_name(lowest_tdate_obj), lowest_tdate_obj[constants.PTR_LINK]))
    print("Most recent transaction_date for {}: {} by {} \n {} ".format(type, highest_tdate, official.get_name(highest_tdate_obj), highest_tdate_obj[constants.PTR_LINK]))
    
    print("Oldest disclosure_date for {}: {} by {} \n {}".format(type, lowest_ddate,  official.get_name(lowest_ddate_obj), lowest_ddate_obj[constants.PTR_LINK]))
    print("Most recent disclosure_date for {}: {} by {}  \n {}\n".format(type, highest_ddate,  official.get_name(highest_ddate_obj), highest_ddate_obj[constants.PTR_LINK]))
    

profile_dates(input_df, constants.INPUT)

### Gender

In [ ]:
def profile_gender(group, normalized=None):
    # d_prime = {'Female' : set(Officials), 'Male' : set(Officials), ...}
    d_prime = {}

    for link, (name, obj) in group.items(): 
        d_prime = dict_utils.increment_set_in_dictionary(d_prime, official.get_gender(name, link), name)

    # d = {'Female' : #_of_officials, 'Male' : #_of_officials, ...}
    d = dict_utils.flatten_len(d_prime, inner_set=True)
    
    if normalized:
        d = dict_utils.normalize(d, normalized, percent=True)
        d = dict_utils.sort_dictionary_by_tuple(d)
                 
    return d

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/profile/gender")

d3 = profile_gender(questionable_people_objects)
csv_utils.make_csv(dir, "profile_gender", d3, [constants.GENDER, constants.INPUT])

d3 = profile_gender(questionable_people_objects, profile_gender(input_all_officials_objects))
csv_utils.make_csv(dir, "profile_gender_normalized", d3, [constants.GENDER, constants.INPUT])

### Party

In [ ]:
def profile_party(group, normalized=None):
    # d_prime = {'Republican' : set(Officials), 'Democrat' : set(Officials), ...}
    d_prime = {}
    
    for (_, off_obj) in group.values(): 
        d_prime = dict_utils.increment_set_in_dictionary(d_prime, off_obj.party, off_obj.name)
    
    # d = {'Republican' : #_of_officials, 'Democrat' : #_of_officials, ...}
    d = dict_utils.flatten_len(d_prime, inner_set=True)
    
    if normalized:         
        d = dict_utils.normalize(d, normalized, percent=True)
        d = dict_utils.sort_dictionary_by_tuple(d)

    return d

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/profile/party")

d1 = profile_party(questionable_people_objects, profile_party(input_all_officials_objects))
csv_utils.make_csv(dir, "profile_party_normalized", (d1), [constants.PARTY, constants.INPUT])

d1 = profile_party(questionable_people_objects)
csv_utils.make_csv(dir, "profile_party", (d1), [constants.PARTY, constants.INPUT])


### State

In [ ]:
def profile_state(group, normalized=None):
    # d_prime = {'Maryland' : set(Officials), 'California' : set(Officials), ...}
    d_prime = {}
    
    # input_officials_objects = {link : (canonical_name_input_based, official_object) ... }
    for _, off_obj in group.values(): 
        d_prime = dict_utils.increment_set_in_dictionary(d_prime, off_obj.state, off_obj.name)

    # d = {'Maryland' : #_of_officials, 'California' : #_of_officials, ...}
    d = dict_utils.flatten_len(d_prime, inner_set=True)
    
    if normalized:
        d = dict_utils.normalize(d, normalized, percent=True)
        d = dict_utils.sort_dictionary_by_tuple(d)
            
    return d 


dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/profile/state")

d1 = profile_state(questionable_people_objects, profile_state(input_all_officials_objects))
csv_utils.make_csv(dir, "profile_state_normalized", (d1), [constants.STATE, constants.INPUT])

d1 = profile_state(questionable_people_objects)
csv_utils.make_csv(dir, "profile_state", (d1), [constants.STATE, constants.INPUT])


### Seniority (Lowest, Highest, Average)

In [ ]:
def profile_seniority(group):
    # d = {x_years_in_congress, y_years_in_congres, ...}
    l = []
    
    # input_officials_objects = {link : (canonical_name_input_based, official_object) ... }
    for _, (_, off_obj) in group.items(): 
        l.append(off_obj.get_seniority())
    
    l.sort()
    
    d = {}
    d["0. Lowest"] = l[0]
    d["1. Average"] = round((sum(l) / len(l)))
    d["2. Highest"] = l[len(l)-1]

    return d 

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/profile/seniority")


d3 = profile_seniority(questionable_people_objects)
csv_utils.make_csv(dir, "profile_seniority", (d3), [constants.SENIORITY, constants.INPUT])


def profile_seniority_normalized(group, normalized=None):
    d = {}
    
    for (_, off_obj) in group.values(): 
        d = dict_utils.increment_dictionary(d, off_obj.get_seniority()) 
    
    if normalized: 
        d_prime = dict_utils.normalize(d, normalized, percent=True)
        return d, d_prime
        
    return d 
    
d3, d6 = profile_seniority_normalized(questionable_people_objects, profile_seniority_normalized(input_all_officials_objects))
csv_utils.make_csv(dir, "profile_seniority", (d3), [constants.SENIORITY, constants.INPUT])
csv_utils.make_csv(dir, "profile_seniority_normalized", (d6), [constants.SENIORITY, constants.INPUT])


def profile_seniority_box(group):
    # d = {x_years_in_congress, y_years_in_congres, ...}
    l = []
    
    # input_officials_objects = {link : (canonical_name_input_based, official_object) ... }
    for _, (_, off_obj) in group.items(): 
        l.append([off_obj.get_seniority()])
    
    return l
    
d1 = profile_seniority_box(questionable_people_objects)
csv_utils.make_csv_base(dir, "profile_seniority_box_input", ["Seniority_Input"], d1)


def profile_seniority_2(group):
    # d = {x_years_in_congress : #_of_people, }
    d = {}
    
    # input_officials_objects = {link : (canonical_name_input_based, official_object) ... }
    for (_, off_obj) in group.values(): 
        d =  dict_utils.increment_dictionary(d, off_obj.get_seniority())
    
    return d 

d3 = profile_seniority_2(questionable_people_objects)


csv_utils.make_csv(dir, "profile_seniority_2", (d3), [constants.SENIORITY, constants.INPUT])

### Get_Congress (Lowest, Highest)

In [ ]:
def profile_congress(group):
    lowest = highest = None 
    lowest_person = highest_person = None 
    
    # input_officials_objects = {link : (canonical_name_input_based, official_object) ... }
    for (name, off_obj) in group.values():
        res = off_obj.get_congress()
        
        if not lowest or res[0] < lowest:
            lowest = res[0]
            lowest_person = name
    
        if not highest or res[len(res) - 1] > highest:
            highest = res[len(res) - 1]
            highest_person = name 
                    
    d = {}
    d["Lowest Congress"] = lowest, lowest_person
    d["Highest Congress"] = highest, highest_person

    return d 
                        
d3 = profile_congress(questionable_people_objects)

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/profile/congress")
csv_utils.make_csv(dir, "profile_congress", (d3), ["", constants.INPUT])

### Number of Degrees (Lowest, Highest, Average)

In [ ]:
# def profile_degrees_2(group, normalized=None):
#     d = {}
#     for (_, off_obj) in group.values(): 
#         d = dict_utils.increment_dictionary(d, off_obj.get_num_of_degrees()) 
    
    
#     if normalized: 
#         d_prime = dict_utils.normalize(d, normalized, percent=True)
#         return d, d_prime 
    
#     return d
    
# d1, d4 = profile_degrees_2(input_house_officials_objects, profile_degrees_2(house_officials_objects))
  
def profile_degrees(group):    
    # d = {x_degrees, y_degrees, ...}
    l = []
    
    # input_officials_objects = {link : (canonical_name_input_based, official_object) ... }
    for (_, off_obj) in group.values(): 
        l.append(off_obj.get_num_of_degrees())
    
    l.sort()
    
    d = {}
    d["0. Lowest"] = l[0]
    d["1. Average"] = round((sum(l) / len(l)))
    d["2. Highest"] = l[len(l)-1]

    return d 
    
d1 = profile_degrees(questionable_people_objects)

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/profile/degrees")
csv_utils.make_csv(dir, "profile_degrees", (d1), ["No. of Degrees", constants.INPUT]) 

def profile_degrees_box(group):    
    # d = {x_degrees, y_degrees, ...}
    l = []
    
    # input_officials_objects = {link : (canonical_name_input_based, official_object) ... }
    for (_, off_obj) in group.values(): 
        l.append([off_obj.get_num_of_degrees()])
    
    return l
    
d2 = profile_degrees_box(questionable_people_objects)
csv_utils.make_csv_base(dir, "profile_degrees_box_input", ["Degrees_input"], d2)

### JD

In [ ]:
def profile_JD(group):
    # d = {x_degrees, y_degrees, ...}
    yes = total = 0 
    
    # input_officials_objects = {link : (canonical_name_input_based, official_object) ... }
    for (_, off_obj) in group.values(): 
        if off_obj.has_JD():
            yes += 1 
        total += 1 
        
    d = {}
    
    d["(Raw, Percent)"] = (yes, round(yes/total, 2))
    
    return d 
    
d3 = profile_JD(questionable_people_objects)        

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/profile/degrees")
csv_utils.make_csv(dir, "profile_JDs", (d3), ["", constants.INPUT]) 


## Transaction Date (transaction_date) 

#### Frequency of Differences between Transaction and Disclosure Date

In [ ]:
def frequency_of_differences(group):
    d = {}
    total = num = 0 


    for _,t in group.iterrows():
        # Negative, X days BEFORE
        # Positive, Y dayas AFTER
        diff = ptr_utils.difference_between_dates(t)      
        total += 1 
        num += diff   
            
        d =  dict_utils.increment_dictionary(d, int(diff))
    
    d["Average"] = round(num/total, 2)

    return d 
    # return dict_utils.sort_dictionary_by_keys(d)

d3 = frequency_of_differences(input_df)

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.TDATE)
csv_utils.make_csv(dir, "frequency_of_differences", (d3), ["Difference", constants.INPUT])

### The most popular transaction_date for each sector.

In [ ]:
def transaction_date_wrt_sector(group, diff):
    # d_prime = {'sector' : {'date' : #_of_transactions, ....} , 'sector2' : .... }
    d_prime = {}
    
    for _,t in group.iterrows():
        if ptr_utils.isvalid(t[constants.TICKER]):
            sector = dir_utils.search_mapping(sector_df, t[constants.TICKER], sector=True)            
            d_prime =  dict_utils.increment_dictionary_in_dictionary(d_prime, sector, (t[constants.TDATE]))
       
    # d = {'ticker' : {'best_date' : #_of_transactions}, .... }
    d = dict_utils.flatten_best(d_prime)

    d = dict_utils.sort_dictionary_by_inner_values(d, reverse=True)

    filename = "transaction_date_wrt_sector"  
    if diff:
        filename += "_" + diff 
    key_header = constants.SECTOR
    value_header = constants.TDATE
    value_header2 = constants.NUMT

    dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.TDATE)
    wd = csv_utils.make_csv(dir, filename, d, [key_header, value_header, value_header2])
    df = pd.read_csv(wd)
    print(df.head(5))
    return d 

_ = transaction_date_wrt_sector(input_df, constants.INPUT)

### The most popular transaction_date for each sector controlling for each official. 

In [ ]:
def transaction_date_wrt_sector_controlled(group, diff):    
    # d_prime = { ('ticker' : {'date' : set(people_who_traded_on_that_day) , ....} , ....}
    d_prime = {}
    
    for _,t in group.iterrows():
        if ptr_utils.isvalid(t[constants.TICKER]):
            sector = dir_utils.search_mapping(sector_df, t[constants.TICKER], sector=True)  

            d_prime = dict_utils.increment_set_in_inner_dictionary(d_prime, sector, t[constants.TDATE], official.get_name(t))
       
    
    d_prime = dict_utils.flatten_len(d_prime)

    # d = {'ticker' : {'best_date' : #_of_transactions}, .... }
    d = dict_utils.flatten_best(d_prime)

    d = dict_utils.sort_dictionary_by_inner_values(d, reverse=True)

    
    filename = "transaction_date_wrt_sector_controlled"
    if diff:
        filename += "_" + diff 
     
    key_header = constants.SECTOR
    value_header = constants.TDATE
    value_header2 = constants.NUMT

    dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.TDATE)
    wd = csv_utils.make_csv(dir, filename, d, [key_header, value_header, value_header2])
    df = pd.read_csv(wd)
    print(df.head(5))
    return d 

_ = transaction_date_wrt_sector_controlled(input_df, constants.INPUT)


### The most popular transaction_date for each industry.

In [ ]:
def transaction_date_wrt_industry(group, diff):    
    # d_prime = {'industry' : {'date' : #_of_transactions, ....} , 'industry2' : .... }
    d_prime = {}
    
    for _,t in group.iterrows():
        if ptr_utils.isvalid(t[constants.TICKER]):
            industry = dir_utils.search_mapping(industry_df, t[constants.TICKER], industry=True)

            d_prime =  dict_utils.increment_dictionary_in_dictionary(d_prime, industry, (t[constants.TDATE]))
       
    # d = {'ticker' : {'best_date' : #_of_transactions}, .... }
    d = dict_utils.flatten_best(d_prime)

    d = dict_utils.sort_dictionary_by_inner_values(d, reverse=True)

    filename = "transaction_date_wrt_industry"
    if diff:
        filename += "_" + diff  
    key_header = constants.INDUSTRY
    value_header = constants.TDATE
    value_header2 = constants.NUMT

    dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.TDATE)
    wd = csv_utils.make_csv(dir, filename, d, [key_header, value_header, value_header2])
    df = pd.read_csv(wd)
    print(df.head(5))
    return d 

_ = transaction_date_wrt_industry(input_df, constants.INPUT)

### The most popular transaction_date for each industry controlling for official. 

In [ ]:
def transaction_date_wrt_industry_controlled(group, diff):
    
    # d_prime = { ('ticker' : {'date' : set(people_who_traded_on_that_day) , ....} , ....}
    d_prime = {}
    
    for _,t in group.iterrows():
        if ptr_utils.isvalid(t[constants.TICKER]):
            industry = dir_utils.search_mapping(industry_df, t[constants.TICKER], industry=True)
            
            d_prime = dict_utils.increment_set_in_inner_dictionary(d_prime, industry, t[constants.TDATE],  official.get_name(t))

    d_prime = dict_utils.flatten_len(d_prime)
    
    # d = {'ticker' : {'best_date' : #_of_transactions}, .... }
    d = dict_utils.flatten_best(d_prime)

    d = dict_utils.sort_dictionary_by_inner_values(d, reverse=True)
    
    filename = "transaction_date_wrt_industry_controlled"
    if diff:
        filename += "_" + diff  
    key_header = constants.INDUSTRY
    value_header = constants.TDATE
    value_header2 = constants.NUMT

    dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.TDATE)
    wd = csv_utils.make_csv(dir, filename, d, [key_header, value_header, value_header2])
    df = pd.read_csv(wd)
    print(df.head(5))
    return d 

_ = transaction_date_wrt_industry_controlled(input_df, constants.INPUT)


### The most popular transaction_date for each ticker. 

In [ ]:
def transaction_date_wrt_ticker(group, diff):    
    # d_prime = {'ticker' : {'date' : #_of_transactions, ....} , 'ticker2' : .... }
    d_prime = {}
    
    for _,t in group.iterrows():
        if ptr_utils.isvalid(t[constants.TICKER]):
            d_prime =  dict_utils.increment_dictionary_in_dictionary(d_prime, t[constants.TICKER], (t[constants.TDATE]))
       
    # d = {'ticker' : {'best_date' : #_of_transactions}, .... }
    d = dict_utils.flatten_best(d_prime)

    d = dict_utils.sort_dictionary_by_inner_values(d, reverse=True)

    filename = "transaction_date_wrt_ticker"
    if diff:
        filename += "_" + diff
    key_header = constants.TICKER
    value_header = constants.TDATE
    value_header2 = constants.NUMT

    dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.TDATE)
    wd = csv_utils.make_csv(dir, filename, d, [key_header, value_header, value_header2])
    df = pd.read_csv(wd)
    print(df.head(5))
    return d 

_ = transaction_date_wrt_ticker(input_df, constants.INPUT)


### The most popular transaction_date for each ticker controlling for official.

In [ ]:
def transaction_date_wrt_ticker_controlled(group, diff):    
    # d_prime = { ('ticker' : {'date' : set(people_who_traded_on_that_day) , ....} , ....}
    d_prime = {}
    
    for _,t in group.iterrows():    
        if ptr_utils.isvalid(t[constants.TICKER]):    
            name = official.get_name(t)
            d_prime = dict_utils.increment_set_in_inner_dictionary(d_prime, t[constants.TICKER], t[constants.TDATE], name)
       
    d_prime = dict_utils.flatten_len_inner_set(d_prime)

    # d = {'ticker' : {'best_date' : #_of_transactions}, .... }
    d = dict_utils.flatten_best(d_prime)
    d = dict_utils.sort_dictionary_by_inner_values(d, reverse=True)

    
    filename = "transaction_date_wrt_ticker_controlled"
    if diff:
        filename += "_" + diff  
        
    key_header = constants.TICKER
    value_header = constants.TDATE
    value_header2 = constants.NUMT

    dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.TDATE)
    wd = csv_utils.make_csv(dir, filename, d, [key_header, value_header, value_header2])
    return d 

_ = transaction_date_wrt_ticker_controlled(input_df, constants.INPUT)

### The most popular transaction_date for type.

In [ ]:
def transaction_date_wrt_type(group, diff):        
    # d_prime = {'type' : {'date' : #_of_transactions, ....} , 'type2' : .... }
    d_prime = {}
    
    for _,t in group.iterrows():
        if ptr_utils.isvalid(t[constants.TICKER]):
            d_prime =  dict_utils.increment_dictionary_in_dictionary(d_prime, ptr_utils.format_type(t[constants.TYPE]), (t[constants.TDATE]))
       
    # d = {'type' : {'best_date' : #_of_transactions}, .... }
    d = dict_utils.flatten_best(d_prime)

    d = dict_utils.sort_dictionary_by_inner_values(d, reverse=True)

    filename = "transaction_date_wrt_type"
    if diff:
        filename += "_" + diff  
    key_header = constants.TYPE
    value_header = constants.TDATE
    value_header2 = constants.NUMT

    dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.TDATE)
    wd = csv_utils.make_csv(dir, filename, d, [key_header, value_header, value_header2])
    df = pd.read_csv(wd)
    print(df)
    return d 

_ = transaction_date_wrt_type(input_df, constants.INPUT)

### The most popular transaction_date for type controlling for official. 

In [ ]:
def transaction_date_wrt_type_controlled(group, diff):    
    # d_prime = { ('type' : {'date' : set(people_who_traded_on_that_day) , ....} , ....}
    d_prime = {}
    
    for _,t in group.iterrows():
        if ptr_utils.isvalid(t[constants.TICKER]):
            d_prime = dict_utils.increment_set_in_inner_dictionary(d_prime, t[constants.TYPE], (t[constants.TDATE]), official.get_name(t))
       

    d_prime = dict_utils.flatten_len(d_prime)
            
    # d = {'type' : {'best_date' : #_of_transactions}, .... }
    d = dict_utils.flatten_best(d_prime)

    d = dict_utils.sort_dictionary_by_inner_values(d, reverse=True)
    
    
    filename = "transaction_date_wrt_type_controlled"
    if diff:
        filename += "_" + diff  
    key_header = constants.TYPE
    value_header = constants.TDATE
    value_header2 = constants.NUMT

    dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.TDATE)
    wd = csv_utils.make_csv(dir, filename, d, [key_header, value_header, value_header2])
    df = pd.read_csv(wd)
    print(df.head(5))
    return d 

_ = transaction_date_wrt_type_controlled(input_df, constants.INPUT)


### The most popular transaction_date for amount.

In [ ]:
def transaction_date_wrt_amount(group, diff):
    # d_prime = {'amount' : {'date' : #_of_transactions, ....} , 'amount1' : .... }
    d_prime = {}
    
    for _,t in group.iterrows():
        if ptr_utils.isvalid(t[constants.TICKER]):
            amount = ptr_utils.consistency_amount(t)
            d_prime =  dict_utils.increment_dictionary_in_dictionary(d_prime, amount, (t[constants.TDATE]))
       
    # d = {'type' : {'best_date' : #_of_transactions}, .... }
    d = dict_utils.flatten_best(d_prime)

    d = dict_utils.add_sort_key_for_amount(d)
    d = dict_utils.sort_dictionary_by_sort_key(d)

    filename = "transaction_date_wrt_amount"
    if diff:
        filename += "_" + diff  
    key_header = constants.TYPE

    dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.TDATE)
    wd = csv_utils.make_csv_breakdown(dir, filename, d, key_header)
    df = pd.read_csv(wd)
    print(df.head(5))
    return d 

_ = transaction_date_wrt_amount(input_df, constants.INPUT)

### The most popular transaction_date for amount controlling for official.

In [ ]:
def transaction_date_wrt_amount_controlled(group, diff):
    # d_prime = { ('amount' : {'date' : set(people_who_traded_on_that_day) , ....} , ....}
    d_prime = {}
    
    for _,t in group.iterrows():
        if ptr_utils.isvalid(t[constants.AMOUNT]):
            amount = ptr_utils.consistency_amount(t)
            d_prime = dict_utils.increment_set_in_inner_dictionary(d_prime, amount, (t[constants.TDATE]), official.get_name(t))
       

    d_prime = dict_utils.flatten_len(d_prime)
        
    # d = {'amount' : {'best_date' : #_of_transactions}, .... }
    d = dict_utils.flatten_best(d_prime)

    d = dict_utils.add_sort_key_for_amount(d)
    d = dict_utils.sort_dictionary_by_sort_key(d)
    
    
    filename = "transaction_date_wrt_amount_controlled"
    if diff:
        filename += "_" + diff  
    key_header = constants.AMOUNT

    dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.TDATE)
    wd = csv_utils.make_csv_breakdown(dir, filename, d, key_header)
    df = pd.read_csv(wd)
    print(df.head(5))
    return d 

_ = transaction_date_wrt_amount_controlled(input_df, constants.INPUT)

### The most popular transaction_date for each official.

In [ ]:
def transaction_date_wrt_official(group, diff):
    # d_prime = {'person1' : {'date' : #_of_transactions, ....} , 'person2' : .... }
    d_prime = {}
    
    for _,t in group.iterrows():
        if ptr_utils.isvalid(t[constants.TICKER]):
            d_prime =  dict_utils.increment_dictionary_in_dictionary(d_prime, official.get_name(t), (t[constants.TDATE]))
       
    # d = {'person' : {'best_date' : #_of_transactions}, .... }
    d = dict_utils.flatten_best(d_prime)


    d = dict_utils.sort_dictionary_by_inner_values(d, reverse=True)

    filename = "transaction_date_wrt_official"
    if diff:
        filename += "_" + diff 

    dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.TDATE)
    wd = csv_utils.make_csv(dir, filename, d, [constants.OFFICIAL, constants.TDATE, constants.NUMT])
    df = pd.read_csv(wd)
    print(df.head(5))
    return d 

transaction_date_wrt_official_res = transaction_date_wrt_official(input_df, constants.INPUT)

### Frequency of Transactions per Date Controlled
_Number of transactions per date controlled by official. E.g. if Ted Baker made 40 transactions on 1/1/02 and Sam Wall made 2 transactions on 1/1/02, we conclude that there were two transactions on 1/1/02._


In [ ]:
def num_of_trans_per_date_controlled(group):    
    d = {}

    for _,t in group.iterrows():
        d =  dict_utils.increment_dictionary_in_dictionary(d, (t[constants.TDATE]), official.get_name(t))

    return dict_utils.flatten_len(d, inner_set=True)
    
d3 = num_of_trans_per_date_controlled(input_df)

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.TDATE)
csv_utils.make_csv(dir, "num_of_trans_per_date_controlled", (d3), [constants.TDATE, constants.INPUT])


### Tax

#### Number of Transactions Within 2 Weeks Prior to Quarterly Tax Date 

In [ ]:
def num_of_trans_within_tax_date(group):        
        total = within = 0 
        d = {}

        for _,t in group.iterrows():  
                total += 1 
                if ptr_utils.within_tax_date(t[constants.TDATE]):
                        within += 1 

        d["(No. of transactions within 2 weeks of tax deadline, %)"] = (within,  round(within/total, 2))

        return d 

d3 = num_of_trans_within_tax_date(input_df)

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.TDATE)
csv_utils.make_csv(dir, "num_of_trans_within_tax_date", (d3), ["",  constants.INPUT])

#### Number of Transactions Within 2 Weeks Prior to Quarterly Tax Date Semi-Controlled 

_Given dict='09/03/2021': {'Thomas H Tuberville': 1, 'Cynthia M Lummis': 1, 'A. Mitchell Mcconnell, Jr.': 1}...I only incremement the number of within (tax date) once per date per official. So, if an official does 100 transactions on a date within two weeks of a quarterly deadline, then I only count it as one transaction._

_A Note: total === number of transactions per person per date (so not really all transactions) because someone could have potentially made 60 transactions on one date which we don't include in neither total or within, if applicable._

In [ ]:
def num_of_trans_within_tax_date_controlled(group):
        total = within = 0 
        people = set()
        d = {}

        for _, t in group.iterrows():
                name = official.get_name(t)
                if ptr_utils.within_tax_date(t[constants.TDATE]) and name not in people:
                        people.add(name)
                        within += 1 
                total += 1         
                
        d["(No. of transactions within 2 weeks of tax deadline, %)"] = (within,  round(within/total, 2))

        return d

d3 = num_of_trans_within_tax_date_controlled(input_df)

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.TDATE)
csv_utils.make_csv(dir, "num_of_trans_within_tax_date_controlled", (d3), ["", constants.INPUT])

## Owner (owner) 

### Frequency Count of Owner

In [ ]:
def freq_count_of_owner(group):
    # d = {'Joint' : 5}
    d = {}

    for _,t in group.iterrows():
        if constants.OWNER in t and ptr_utils.isvalid(t[constants.OWNER]):                
            d =  dict_utils.increment_dictionary(d, t[constants.OWNER].capitalize())
        
    return dict_utils.sort_dictionary_by_values(d)

# {link : (canonical_name_input_based, official_object) ... }
d3 = freq_count_of_owner(input_df)

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.OWNER)
csv_utils.make_csv(dir, "freq_count_of_owner", (d3), ["Owner", constants.INPUT]) 

In [ ]:
def freq_count_by_spouse(group):
    # d = {'x_spouse' : 5}
    d = {}

    for _,t in group.iterrows():
        if constants.OWNER in t and ptr_utils.isvalid(t[constants.OWNER]) and t[constants.OWNER].capitalize() == 'Spouse':
            d =  dict_utils.increment_dictionary(d, official.get_name(t))
        
    return d
    
    
# {link : (canonical_name_input_based, official_object) ... }
d3 = freq_count_by_spouse(input_df)

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.OWNER)
csv_utils.make_csv(dir, "freq_count_by_spouse", (d3), ["Owner", constants.INPUT]) 

## Ticker (ticker) 

### Number of Tickers

In [ ]:
def num_of_tickers(group):
    # d = {'ticker' : #_of_times }
    d = {}
    
    for _,t in group.iterrows():
        if ptr_utils.isvalid(t[constants.TICKER]):
            d =  dict_utils.increment_dictionary(d, t[constants.TICKER])
       

    return dict_utils.sort_dictionary_by_values(d, reverse=True)

d3 = num_of_tickers(input_df)
    
dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.TICKER)
csv_utils.make_csv(dir, "num_of_tickers", (d3), [constants.TICKER,  constants.INPUT])

### Frequency of Ticker per Year

In [ ]:
def frequency_of_ticker_breakdown_year(group, diff):
    d = {}

    # {"ticker" : {"year" : number, "year" : number, ...}}
    for _,t in group.iterrows():
        if ptr_utils.isvalid(t[constants.TICKER]):
            d = dict_utils.increment_dictionary_in_dictionary(d, t[constants.TICKER], ptr_utils.get_year(t[constants.TDATE]))
            d = dict_utils.increment_dictionary_in_dictionary(d, t[constants.TICKER], 9999)

    d = dict_utils.sort_dictionary_by_keys(d)
    filename = "frequency_of_ticker_breakdown_year"
    if diff:
        filename += "_" + diff
    key_header = constants.TICKER
    
    
    dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.TICKER)
    wd = csv_utils.make_csv_breakdown(dir, filename, d, key_header)
    print(pd.read_csv(wd).head(2))

frequency_of_ticker_breakdown_year(input_df, constants.INPUT)

### Frequency of Ticker per Date

In [ ]:
def frequency_of_ticker_by_date(group, diff):
    # {ticker : {date : ___}}
    d = {}
    
    for _,t in group.iterrows():
        if ptr_utils.isvalid(t[constants.TICKER]):
            d = dict_utils.increment_dictionary_in_dictionary(d, t[constants.TICKER], t[constants.TDATE])

    
    d = dict_utils.flatten_best(d)
    

    filename = "frequency_of_ticker_by_date"
    if diff:
        filename += "_" + diff
    
    dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.TICKER)
    wd = csv_utils.make_csv_breakdown(dir, filename, d, constants.TICKER)
    print(pd.read_csv(wd).head(2))

d3 = frequency_of_ticker_by_date(input_df, constants.INPUT)

## Industry & Sector 

#### Number of Transactions per Industry
_Not controlled in any way._

In [ ]:
def number_of_transactions_per_indusry(group):        
    d = {}
    
    for _,t in group.iterrows():
        if ptr_utils.isvalid(t[constants.TICKER]):
            industry = dir_utils.search_mapping(industry_df, t[constants.TICKER], industry=True)
            d = dict_utils.increment_dictionary(d, industry)

    return dict_utils.sort_dictionary_by_values(d)
    

d3 = number_of_transactions_per_indusry(input_df)

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.INDUSTRY)
csv_utils.make_csv(dir, "number_of_transactions_per_indusry", (d3), [constants.INDUSTRY, constants.INPUT])



def number_of_transactions_per_sector(group):        
    d = {}
    
    for _,t in group.iterrows():
        if ptr_utils.isvalid(t[constants.TICKER]):
            sector = dir_utils.search_mapping(sector_df, t[constants.TICKER], sector=True)
            d = dict_utils.increment_dictionary(d, sector)

    return dict_utils.sort_dictionary_by_values(d)
    

d3 = number_of_transactions_per_sector(input_df)

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.SECTOR)
csv_utils.make_csv(dir, "number_of_transactions_per_sector", (d3), [constants.SECTOR, constants.INPUT])

#### Industry Breakdown per Official

In [ ]:
def frequency_of_industry_breakdown_official(group, diff):
    d = {}

    for _,t in group.iterrows():
        if ptr_utils.isvalid(t[constants.TICKER]):
            industry = dir_utils.search_mapping(industry_df, t[constants.TICKER], industry=True)
            d = dict_utils.increment_dictionary_in_dictionary(d, official.get_name(t), industry)

    d = dict_utils.sort_dictionary_by_keys(d)
    
    filename = "frequency_of_industry_breakdown_official"
    if diff:
        filename += "_" + diff
    dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.INDUSTRY)

    wd = csv_utils.make_csv_breakdown(dir, filename, d, constants.OFFICIAL)
    print(pd.read_csv(wd).head(2))
    
    return d 

d3 = frequency_of_industry_breakdown_official(input_df, constants.INPUT)



def frequency_of_sector_breakdown_official(group, diff):
    d = {}

    for _,t in group.iterrows():
        if ptr_utils.isvalid(t[constants.TICKER]):
            sector = dir_utils.search_mapping(sector_df, t[constants.TICKER], sector=True)
            d = dict_utils.increment_dictionary_in_dictionary(d, official.get_name(t), sector)

    d = dict_utils.sort_dictionary_by_keys(d)
    
    filename = "frequency_of_sector_breakdown_official"
    if diff:
        filename += "_" + diff
    dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.SECTOR)

    wd = csv_utils.make_csv_breakdown(dir, filename, d, constants.OFFICIAL)
    print(pd.read_csv(wd).head(2))
    
    return d 

d3 = frequency_of_sector_breakdown_official(input_df, constants.INPUT)

#### Frequency of Industry per Year

In [ ]:
def frequency_of_industry_breakdown(group, diff):
    d = {}

    for _,t in group.iterrows():     
        if ptr_utils.isvalid(t[constants.TICKER]):
            industry = dir_utils.search_mapping(industry_df, t[constants.TICKER], industry=True)  
            d = dict_utils.increment_dictionary_in_dictionary(d, industry, ptr_utils.get_year(t[constants.TDATE]))

    d = dict_utils.sort_dictionary_by_keys(d)
    
    filename = "frequency_of_industry_breakdown"
    if diff:
        filename += "_" + diff 
    key_header = "industry"
    
    
    dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.INDUSTRY)
    wd = csv_utils.make_csv_breakdown(dir, filename, d, key_header)
    print(pd.read_csv(wd).head(2))
    return d 

d3 = frequency_of_industry_breakdown(input_df, constants.INPUT)


def frequency_of_sector_breakdown(group, diff):
    d = {}

    for _,t in group.iterrows():     
        if ptr_utils.isvalid(t[constants.TICKER]):
            sector = dir_utils.search_mapping(sector_df, t[constants.TICKER], sector=True)  
            d = dict_utils.increment_dictionary_in_dictionary(d, sector, ptr_utils.get_year(t[constants.TDATE]))

    d = dict_utils.sort_dictionary_by_keys(d)
    
    filename = "frequency_of_sector_breakdown"
    if diff:
        filename += "_" + diff 
    key_header = "sector"
    
    
    dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.SECTOR)
    wd = csv_utils.make_csv_breakdown(dir, filename, d, key_header)
    print(pd.read_csv(wd).head(2))
    return d 

d3 = frequency_of_sector_breakdown(input_df, constants.INPUT)

## Asset Description (asset_description) and Comment (comment) 

### Number of Options

In [ ]:
def number_of_options(group):
    count = total = 0 
    d = {}
    # [this_person_placed_an_option, ...]
    people = set()

    for _,t in group.iterrows():
        if constants.ASSET_DESCRIPTION in t and ptr_utils.isvalid(t[constants.ASSET_DESCRIPTION]) and ("Put" in t[constants.ASSET_DESCRIPTION]  or "put" in t[constants.ASSET_DESCRIPTION]  or "Call" in t[constants.ASSET_DESCRIPTION]  or "call" in t[constants.ASSET_DESCRIPTION]  or "Option" in t[constants.ASSET_DESCRIPTION] or "option" in t[constants.ASSET_DESCRIPTION]): 
            count += 1 
            people.add(official.get_name(t))
        total += 1 

    d["(No. of Options, %)"] = (count, round(count/total, 2))

    return d 

d3 = number_of_options(input_df)

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.COMMENT)
csv_utils.make_csv(dir, "number_of_options", (d3), ["",  constants.INPUT])    

### Number of Scanned PDFs.

In [ ]:
def number_of_scanned_pdfs(group):
    count = total = 0 
    d = {}

    for _,t in group.iterrows():
        if t[constants.ASSET_DESCRIPTION] == constants.DISCLOSED:
            count += 1 
        total += 1 
            
    d["(No. of Scanned PDFS, %)"] = (count, round(count/total, 2))

    return d 
            
d3 = number_of_scanned_pdfs(input_df)

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.COMMENT)
csv_utils.make_csv(dir, "number_of_scanned_pdfs", (d3), ["", constants.INPUT])    

## Asset Type (asset_type)

### Frequency of Asset Type

In [ ]:
def frequency_of_asset_type(group):
    d = {}

    for _,t in group.iterrows():
        if constants.ATYPE in t and ptr_utils.isvalid(t[constants.ATYPE]):
            d = dict_utils.increment_dictionary(d, t[constants.ATYPE])
      
    return dict_utils.sort_dictionary_by_values(d)

d3 = frequency_of_asset_type(input_df)

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.ATYPE)
csv_utils.make_csv(dir, "frequency_of_asset_type", (d3), [constants.ATYPE, constants.INPUT])

## Amount 

### Frequency of Amount by Person

In [ ]:
def frequency_of_amount_by_persom(group, diff):
    d = {}

    for _,t in group.iterrows():
        amount = ptr_utils.consistency_amount(t)
        d = dict_utils.increment_dictionary_in_dictionary(d, official.get_name(t), amount)
    
    d = dict_utils.sort_dictionary_by_keys(d)
    
    filename = "frequency_of_amount_by_person"
    if diff:
        filename += "_" + diff 
        
    key_header = constants.AMOUNT
    
    dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.AMOUNT)
    wd = csv_utils.make_csv_breakdown(dir, filename, d, key_header)
    print(pd.read_csv(wd).head(2))
    
    return d 
   
    
d2 = frequency_of_amount_by_persom(input_df, constants.INPUT)

### Frequency of Transactions by Amount

In [ ]:
def frequency_of_amount_total(group):
    d = {}        

    for _,t in group.iterrows():
        amount = ptr_utils.consistency_amount(t)                    
        d = dict_utils.increment_dictionary(d, amount)

    return d
    
d3 = frequency_of_amount_total(input_df)


dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.AMOUNT)
csv_utils.make_csv(dir, "frequency_of_amount_total", (d3), [constants.AMOUNT, constants.INPUT])

### Frequency of Amount by Gender 

In [ ]:
# def frequency_of_amount_by_gender(group, diff, normalized=None):
#     d = {}
    
#     for _,t in group.iterrows():
#         name = official.get_name(t)
#         link = input_all_officials_name[name]
#         amount = ptr_utils.consistency_amount(t)
#         d = dict_utils.increment_dictionary_in_dictionary(d, amount, all_officials_gender[link])

#     if normalized: 
#         for amount in d:
#             for gender in d[amount]:
#                 d[amount][gender] /= normalized[gender]
        
#     d = dict_utils.add_sort_key_for_amount(d)
#     d = dict_utils.sort_dictionary_by_sort_key(d)


#     filename = "frequency_of_amount_by_gender"
#     if diff:
#         filename += "_" + diff
#     if normalized:
#         filename += "_normalized"
    
#     dir = dir_utils.makesubdir(constants.path_csv, constants.AMOUNT)
#     wd = csv_utils.make_csv_breakdown(dir, filename, d, constants.AMOUNT)
#     print(pd.read_csv(wd).head(2))
#     return d 
   
# d1 = frequency_of_amount_by_gender(house_input_df, constants.HOUSE, profile_gender(input_house_officials_link))
# d2 = frequency_of_amount_by_gender(senate_input_df, constants.SENATE, profile_gender(input_senate_officials_link))
# d3 = frequency_of_amount_by_gender(input_df, constants.INPUT, profile_gender(input_all_officials_link))
    
# d1 = frequency_of_amount_by_gender(house_input_df, constants.HOUSE)
# d2 = frequency_of_amount_by_gender(senate_input_df, constants.SENATE)
# d3 = frequency_of_amount_by_gender(input_df, constants.INPUT)

### Frequency of Transactions by Political Affiliation and Amount

In [ ]:
# def frequency_of_amount_by_aff(group, diff):
#     d = {}

#     for _,t in group.iterrows():
#         obj = t_to_obj(t)
#         amount = ptr_utils.consistency_amount(t)            
#         d = dict_utils.increment_dictionary_in_dictionary(d, amount, obj.party)


#     d = dict_utils.add_sort_key_for_amount(d)
#     d = dict_utils.sort_dictionary_by_sort_key(d)
    
        
#     filename = "frequency_of_amount_by_aff"
#     if diff:
#         filename += "_" + diff
    
#     dir = dir_utils.makesubdir(constants.path_csv, constants.AMOUNT)
#     wd = csv_utils.make_csv_breakdown(dir, filename, d, constants.AMOUNT)
#     print(pd.read_csv(wd).head(2))
    
#     return d 
   
    
# d1 = frequency_of_amount_by_aff(house_input_df, constants.HOUSE)
# d2 = frequency_of_amount_by_aff(senate_input_df, constants.SENATE)
# d3 = frequency_of_amount_by_aff(input_df, constants.INPUT) 

### Average Activity

In [ ]:
# def average_activity(group, diff):
#     d={}
    
#     for _,t in group.iterrows():
#         if ptr_utils.isvalid(t[constants.AMOUNT]):
                        
#             mean = ptr_utils.get_gmean(t[constants.AMOUNT])

#             d = dict_utils.increment_list_in_dictionary(d, official.get_name(t), mean)


#     d = dict_utils.flatten_gmean(d)        
            
#     filename = "average_activity"
#     if diff:
#         filename += "_" + diff

#     key_header = constants.OFFICIAL 
#     value_header = "average_size_of_transactions"

#     d = dict_utils.sort_dictionary_by_values(d)
#     d = dict_utils.commify(d)

#     dir = dir_utils.makesubdir(constants.path_csv, constants.AMOUNT)
#     wd = csv_utils.make_csv(dir, filename, d, [key_header, value_header])
#     df = pd.read_csv(wd)
#     print(df.head(5))

#     return d 

# d1 = average_activity(house_input_df, constants.HOUSE)
# d2 = average_activity(senate_input_df, constants.SENATE)
# d3 = average_activity(input_df, constants.INPUT) 

## Types (type)

### Types of Actions Total

In [ ]:
def frequency_of_act(group):
    d = {}

    for _,t in group.iterrows():
        if ptr_utils.isvalid(t[constants.TYPE]):
            d = dict_utils.increment_dictionary(d, t[constants.TYPE])
    
    return dict_utils.sort_dictionary_by_values(d)
    
     
d3 = frequency_of_act(input_df)

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.TYPE)
csv_utils.make_csv(dir, "frequency_of_act", (d3), [constants.TYPE,  constants.INPUT])

### Types of Transactions per Person

In [ ]:
def types_of_transactions_per_person(group, diff, normalized=None):
    d={}
    
    for _,t in group.iterrows():
        d = dict_utils.increment_dictionary_in_dictionary(d, official.get_name(t), t[constants.TYPE])
        d = dict_utils.increment_dictionary_in_dictionary(d, official.get_name(t), constants.TOTAL)


    if normalized: 
        for k,v in d.items():
            newinner = {}
            for ik, iv in v.items():
                link = input_all_officials_name[k]
                _, obj = input_all_officials_objects[link]
                newinner[ik] = round(iv/obj.get_seniority(), 2)
                
            d[k] = newinner
    
    filename = "types_of_transactions_per_person"
    if diff:
        filename += "_" + diff
    if normalized:
        filename += "_normalized"

    d = dict_utils.sort_dictionary_by_keys(d)

    dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.TYPE)
    wd = csv_utils.make_csv_breakdown(dir, filename, d, constants.OFFICIAL)
    df = pd.read_csv(wd)
    print(df.head(5))

    return d 

d3 = types_of_transactions_per_person(input_df, constants.INPUT)

d3 = types_of_transactions_per_person(input_df, constants.INPUT, normalized=True)

## Frequency

### Number of Transactions per Year

In [ ]:
def num_of_trans_per_year(group, normalized=None):
    d={}
    
    for _,t in group.iterrows():
        d =  dict_utils.increment_dictionary(d, ptr_utils.get_year(t[constants.TDATE]))

    if normalized:
        d2 = {}
        for k,v in d.items():
            d2[k] = v/normalized
            
        
    return dict_utils.sort_dictionary_by_values(d), dict_utils.sort_dictionary_by_values(d2)

d3, d6 = num_of_trans_per_year(input_df, len(input_all_officials_objects))

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.FREQ)
csv_utils.make_csv(dir, "num_of_trans_per_year", (d3), ["year", constants.INPUT])
csv_utils.make_csv(dir, "num_of_trans_per_year_normalized", d6, ["year", constants.INPUT])

### Number of Transactions per Person 
_Not controlling for number of years in position or size of transaction._ For each official, we want their total number of transactions.  {'Sam': 5, 'Alex': 2424, ...}

In [ ]:
def num_of_trans_per_person(group):
    d={}
    
    for _,t in group.iterrows():
        d = dict_utils.increment_dictionary(d, official.get_name(t))

    return dict_utils.sort_dictionary_by_values(d)
    
d3 = num_of_trans_per_person(input_df)

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.PROFILE)
csv_utils.make_csv(dir, "num_of_trans_per_person", (d3), [constants.OFFICIAL, constants.INPUT])


def num_of_trans_per_person_normalized(group):
    d={}
    
    for _,t in group.iterrows():
        d = dict_utils.increment_dictionary(d, official.get_name(t))

    new_d = {}
    for k,v in d.items():
        link = input_all_officials_name[k]
        _, obj = input_all_officials_objects[link]
        new_d[k] = round(v/obj.get_seniority(), 2) if round(v/obj.get_seniority(), 2) != 0 else 1 
        
    return new_d
    

d3 = num_of_trans_per_person_normalized(input_df)

dir = dir_utils.makesubdir(constants.path_csv, "insider_trading/"+constants.PROFILE)
csv_utils.make_csv(dir, "num_of_trans_per_person_normalized", (d3), [constants.OFFICIAL, constants.INPUT])